In [ ]:
import tensorflow as tf
from keras import layers

In [ ]:
# simple autoecnoder

class Simple_Autoencoder:
    def __init__(self, input_size, num_hidden_layers, hidden_layer_size, bottleneck_size):
        self.input_size = input_size
        self.output_size = input_size
        self.bottleneck_size = bottleneck_size
        self.num_hidden_layers = num_hidden_layers
        self.hidden_layer_size = hidden_layer_size

        self.optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
        self.loss = tf.keras.losses.MeanSquaredError()

        # build

        return